# Problems

## 7)
> We want to do #2 for minimal error. Using this method, more centroids will aapear in the less dense region and accordingly, fewer centroids will appear in the denser regions. Having areas where points are far apart causes more centroids adn therefore the net distance from centreoid would decrease and provide more accurate calculations.

## 11)
> Having a low SSE for all clusters would mean that the variable is treated like a constant and wouldn't be useful for dividing data into subgroups.
<br>Having a high SSE for all clusters would result in more noise than meaningful data.
<br>If the value is low for only one cluster, than it defines the cluster itself.
<br>If the value is high fro only one cluster than it's not providing much information.
<br>Per Variable SSE information can be used to improve clustering as poor performing variables can be removed if they don't differentiate clusters well. Outliers can be removed via Per Variable SSE.

## 16)
>Diagram for Part A:
<br>![](./diagrams/16_a.png)
>Diagram for Part B:
<br>![](./diagrams/16_b.png)


## 17)
>a.1)<br>First Cluster: {6, 12, 18, 24, 30}
<br>Error First: (12<sup>2</sup> + 6<sup>2</sup> + 0<sup>2</sup> + 6<sup>2</sup> + 12<sup>2</sup>) = 360
<br>Second Cluster: {42, 48}.
<br>Error Second: (3<sup>2</sup> + 3<sup>2</sup>) = 18.
<br>Total Error: 378
<br>a.2)<br>First Cluster: {6, 12, 18, 24}
<br>Error First: 9<sup>2</sup> + 3<sup>2</sup> + 3<sup>2</sup> + 9<sup>2</sup>) = 180
<br>Second Cluster: {30, 42, 48}
<br>Error Second: (10<sup>2</sup> + 2<sup>2</sup> + 8<sup>2</sup>) = 168. Total error = 180 + 168 = 348
<br>b) Both Centroids are stable.
<br>c) {6, 12, 18, 24, 30} and {42, 48}.
<br>d) Single link produces more "natural" clustering.
<br>e) Single link results in contiguous clusters as it goes with denser groups.
<br>f)K-means struggles to find clusters of different sizes because it tries to minimize squared error, resulting in larger, less natural, clusters.

## 21)
><br>Entropy of C1: 0.20
<br>Entropy of C2: 1.84
<br>Entropy of C3: 1.69
<br>
<br>Purity of C1: 676/693 = 0.98
<br>Purity of C2: 827/1562 = 0.53
<br>Purity of C3: 465/949 = 0.49
<table>
  <tr>
    <th>Cluster #</th>
    <th>Entropy</th>
    <th>Purity</th>
  </tr><tr>
    <td>1</td>
    <td>0.20</td>
    <td>0.98</td>
  </tr><tr>
    <td>2</td>
    <td>1.84</td>
    <td>0.53</td>
  </tr><tr>
    <td>3</td>
    <td>1.70</td>
    <td>0.49</td>
</table>


## 22)
><br>a) There is a difference because they will follow different distributions. Uniformly spaced points will have uniform density unlike points randomly generated from a uniform distribution which will have varying density.
<br>b) The Random Set.
<br>c) DBSCAN will either merge the points into a single cluster or identify them as noise for uniform data. for the random data, it may find multiple clusters in the variations of point density.


# Practicum

## 2.1

In [59]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

#Names Attributes
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
all_df = pd.read_csv("./Data_Sets/auto-mpg.data", delim_whitespace=True, names=column_names)
auto_df = all_df[['mpg','displacement','horsepower','weight','acceleration']]

#Finds missing HP's and replaces with numpy NaN
missing_hp = auto_df[auto_df.horsepower == '?'].index
auto_df.loc[missing_hp]
auto_df.loc[missing_hp, 'horsepower'] = np.nan
auto_df.horsepower  = auto_df.horsepower.apply(pd.to_numeric)

mean_imputer =   SimpleImputer(missing_values = np.nan, strategy = 'mean')

# Impute with mean and print summary
imputed_mean_df = auto_df.copy()
imputed_mean_df['horsepower'] = (mean_imputer.fit_transform(auto_df[['horsepower']])).ravel()


#print("\n----Mean Replaced Summary Statistics----\n")
#print(imputed_mean_df.describe())
#print("\nMean Replaced Variance: " + str(imputed_mean_df.loc[:,'horsepower'].var()))

In [60]:
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(linkage='average',n_clusters=3,)
X = imputed_mean_df.values
y = imputed_mean_df.columns.values
clusters = clustering.fit(X)
labels = clustering.labels_
clusters

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='average', memory=None, n_clusters=3)

In [61]:
print("Number of clusters: " + str(clusters.n_clusters_))

Number of clusters: 3


In [62]:
cluster_1 = X[labels==0]
cluster_2 = X[labels==1]
cluster_3 = X[labels==2]
#print("Cluster 1 Size: " +str(len(cluster_1)))
#print("Cluster 2 Size: " +str(len(cluster_2)))
#print("Cluster 3 Size: " +str(len(cluster_3)))
#print("Sum: " + str(len(cluster_1)+len(cluster_2)+len(cluster_3)))
c1_means = list(cluster_1.mean(axis=0))
c1_means.insert(0,1) # Add cluster number to front
c2_means = list(cluster_2.mean(axis=0))
c2_means.insert(0,2) # Add cluster number to front
c3_means = list(cluster_3.mean(axis=0))
c3_means.insert(0,3) # Add cluster number to front

mean_df = pd.DataFrame([c1_means,c2_means,c3_means],columns =['cluster','mpg','displacement','horsepower','weight','acceleration'])
#mean_df.concat([c2_means])
print("Means of Clusters:")
print(mean_df.to_string(index=False))


cluster        mpg  displacement  horsepower       weight  acceleration
       1  27.365414    131.934211   84.300061  2459.511278     16.298120
       2  13.889062    358.093750  167.046875  4398.593750     13.025000
       3  17.510294    278.985294  124.470588  3624.838235     15.105882


In [63]:
c1_vars = list(cluster_1.var(axis=0))
c1_vars.insert(0,1) # Add cluster number to front
c2_vars = list(cluster_2.var(axis=0))
c2_vars.insert(0,2) # Add cluster number to front
c3_vars = list(cluster_3.var(axis=0))
c3_vars.insert(0,3) # Add cluster number to front

variance_df = pd.DataFrame([c1_vars,c2_vars,c3_vars],columns =['cluster','mpg','displacement','horsepower','weight','acceleration'])
#mean_df.concat([c2_means])
print("Variances of Clusters:")
print(variance_df.to_string(index=False))

Variances of Clusters:
 cluster        mpg  displacement  horsepower         weight  acceleration
       1  41.818503   2817.451499  367.755734  181945.513031      5.696801
       2   3.306599   2104.803711  744.700928   73151.209961      3.535313
       3   8.700041   2840.102725  702.602076   37220.282656     10.401730


### Now Include Origin Feature

In [55]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

#Names Attributes
column_names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
all_df = pd.read_csv("./Data_Sets/auto-mpg.data", delim_whitespace=True, names=column_names)
auto2_df = all_df[['mpg','displacement','horsepower','weight','acceleration','origin']]

#Finds missing HP's and replaces with numpy NaN
missing_hp = auto_df[auto_df.horsepower == '?'].index
auto2_df.loc[missing_hp]
auto2_df.loc[missing_hp, 'horsepower'] = np.nan
auto2_df.horsepower  = auto_df.horsepower.apply(pd.to_numeric)

mean_imputer =   SimpleImputer(missing_values = np.nan, strategy = 'mean')

# Impute with mean and print summary
imputed_mean_df2 = auto2_df.copy()
imputed_mean_df2['horsepower'] = (mean_imputer.fit_transform(auto2_df[['horsepower']])).ravel()


In [49]:
from sklearn.cluster import AgglomerativeClustering
clustering2 = AgglomerativeClustering(linkage='average',n_clusters=3,)
X2 = imputed_mean_df2.values
y2 = imputed_mean_df2.columns.values
clusters2 = clustering2.fit(X2)
labels2 = clustering2.labels_
clusters

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='average', memory=None, n_clusters=3)

In [64]:
print("Number of clusters: " + str(clusters2.n_clusters_))

Number of clusters: 3


In [54]:
cluster_12 = X2[labels==0]
cluster_22 = X2[labels==1]
cluster_32 = X2[labels==2]
#print("Cluster 1 Size: " +str(len(cluster_1)))
#print("Cluster 2 Size: " +str(len(cluster_2)))
#print("Cluster 3 Size: " +str(len(cluster_3)))
#print("Sum: " + str(len(cluster_1)+len(cluster_2)+len(cluster_3)))
c1_means2 = list(cluster_12.mean(axis=0))
c1_means2.insert(0,1) # Add cluster number to front
c2_means2 = list(cluster_22.mean(axis=0))
c2_means2.insert(0,2) # Add cluster number to front
c3_means2= list(cluster_32.mean(axis=0))
c3_means2.insert(0,3) # Add cluster number to front

mean_df2 = pd.DataFrame([c1_means2,c2_means2,c3_means2],columns =['cluster','mpg','displacement','horsepower','weight','acceleration','origin'])
#mean_df.concat([c2_means])
print("Means of Clusters:")
print(mean_df2.to_string(index=False))

cluster        mpg  displacement  horsepower       weight  acceleration    origin
       1  27.365414    131.934211   84.300061  2459.511278     16.298120  1.845865
       2  13.889062    358.093750  167.046875  4398.593750     13.025000  1.000000
       3  17.510294    278.985294  124.470588  3624.838235     15.105882  1.044118


Compare to the clusters without origin calculated previously

In [65]:
print(mean_df.to_string(index=False))

cluster        mpg  displacement  horsepower       weight  acceleration
       1  27.365414    131.934211   84.300061  2459.511278     16.298120
       2  13.889062    358.093750  167.046875  4398.593750     13.025000
       3  17.510294    278.985294  124.470588  3624.838235     15.105882


We can see that the clusters are identical, and cluster assignment is not affected by class label.

In [67]:
c1_vars2 = list(cluster_12.var(axis=0))
c1_vars2.insert(0,1) # Add cluster number to front
c2_vars2 = list(cluster_22.var(axis=0))
c2_vars2.insert(0,2) # Add cluster number to front
c3_vars2 = list(cluster_32.var(axis=0))
c3_vars2.insert(0,3) # Add cluster number to front

variance_df2 = pd.DataFrame([c1_vars2,c2_vars2,c3_vars2],columns =['cluster','mpg','displacement','horsepower','weight','acceleration','origin'])
#mean_df.concat([c2_means])
print("Variances of Clusters:")
print(variance_df2.to_string(index=False))

Variances of Clusters:
 cluster        mpg  displacement  horsepower         weight  acceleration    origin
       1  41.818503   2817.451499  367.755734  181945.513031      5.696801  0.724363
       2   3.306599   2104.803711  744.700928   73151.209961      3.535313  0.000000
       3   8.700041   2840.102725  702.602076   37220.282656     10.401730  0.042171


Again, let's compare variances

In [68]:
c1_vars = list(cluster_1.var(axis=0))
c1_vars.insert(0,1) # Add cluster number to front
c2_vars = list(cluster_2.var(axis=0))
c2_vars.insert(0,2) # Add cluster number to front
c3_vars = list(cluster_3.var(axis=0))
c3_vars.insert(0,3) # Add cluster number to front

variance_df = pd.DataFrame([c1_vars,c2_vars,c3_vars],columns =['cluster','mpg','displacement','horsepower','weight','acceleration'])
#mean_df.concat([c2_means])
print("Variances of Clusters:")
print(variance_df.to_string(index=False))

Variances of Clusters:
 cluster        mpg  displacement  horsepower         weight  acceleration
       1  41.818503   2817.451499  367.755734  181945.513031      5.696801
       2   3.306599   2104.803711  744.700928   73151.209961      3.535313
       3   8.700041   2840.102725  702.602076   37220.282656     10.401730


It's identical as would be expected with the means being identical.

## 2.2

In [88]:
from sklearn.datasets import load_boston
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd

boston = load_boston()
df = pd.DataFrame(data=boston.data, columns=boston.feature_names)
df_scaled = pd.DataFrame(data=preprocessing.scale(boston.data), columns=boston.feature_names)

In [109]:
model2 = KMeans(n_clusters=2, init='k-means++')
labels2 = model2.fit_predict(df_scaled)
silhouette = metrics.silhouette_score(df_scaled, labels2)

print("Silhouette Score for 2 Clusters:")
print(silhouette)

Silhouette Score for 2 Clusters:
0.36011768587358606


In [104]:
model3 = KMeans(n_clusters=3, init='k-means++')
labels3 = model3.fit_predict(df_scaled)
silhouette = metrics.silhouette_score(df_scaled, labels3)

print("Silhouette Score for 3 Clusters:")
print(silhouette)

Silhouette Score for 3 Clusters:
0.2574894522739468


In [105]:
model4 = KMeans(n_clusters=4, init='k-means++')
labels4 = model.fit_predict(df_scaled)
silhouette = metrics.silhouette_score(df_scaled, labels4)

print("Silhouette Score for 4 Clusters:")
print(silhouette)

Silhouette Score for 4 Clusters:
0.36011768587358606


In [106]:
model5 = KMeans(n_clusters=5, init='k-means++')
labels5 = model5.fit_predict(df_scaled)
silhouette = metrics.silhouette_score(df_scaled, labels5)

print("Silhouette Score for 5 Clusters:")
print(silhouette)

Silhouette Score for 5 Clusters:
0.24783915229552517


In [107]:
model6 = KMeans(n_clusters=6, init='k-means++')
labels6 = model6.fit_predict(df_scaled)
silhouette = metrics.silhouette_score(df_scaled, labels6)

print("Silhouette Score for 6 Clusters:")
print(silhouette)

Silhouette Score for 6 Clusters:
0.28497980790216176


In [118]:
optimalMeans1 = list(np.mean(df_scaled[labels2 == 0], axis =0))
optimalMeans2 = list(np.mean(df_scaled[labels2 == 1], axis =0))

optimalMeans1.insert(0,1) # Add cluster number to front
optimalMeans2.insert(0,2) # Add cluster number to front

column_names = list(boston.feature_names)
mean_df3 = pd.DataFrame([optimalMeans1, optimalMeans2], columns=['Cluster']+ column_names)

print("Average values for each cluster where n_cluster = 2\n")
print(mean_df3.to_string(index=False))


Average values for each cluster where n_cluster = 2

 Cluster      CRIM        ZN     INDUS      CHAS       NOX        RM       AGE       DIS       RAD       TAX   PTRATIO         B     LSTAT
       1 -0.390124  0.262392 -0.620368  0.002912 -0.584675  0.243315 -0.435108  0.457222 -0.583801 -0.631460 -0.285808  0.326451 -0.446421
       2  0.725146 -0.487722  1.153113 -0.005412  1.086769 -0.452263  0.808760 -0.849865  1.085145  1.173731  0.531248 -0.606793  0.829787


Based on gathered data, the optimal number of clusters was 2, with 3 being the worst. The mean values of each feature for both clusters is shown above.

## 2.3

In [125]:
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import completeness_score
from sklearn.datasets import load_wine
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn import metrics
import pandas as pd

wine = load_wine()
df = pd.DataFrame(data=wine.data, columns=wine.feature_names)
df_scaled = pd.DataFrame(data=preprocessing.scale(wine.data), columns=wine.feature_names)
wine_model = KMeans(n_clusters=3, init='k-means++')
wine_labels = model.fit_predict(df)


In [124]:
target_values = wine.target
target_labels = pd.qcut(x=target_values,q=2,labels=[1,2])
class_labels = target_labels.astype('int32') - 1

print("Homogeneity Score:")
print(homogeneity_score(class_labels, wine_labels))

print("\nCompleteness Score:")
print(completeness_score(class_labels, wine_labels))

Homogeneity Score:
0.1634165118531094

Completeness Score:
0.1540691568070359


In [ ]:
Homogeneity Score describes what percentage of clusters belonging to a single class. Because of this, the score is independent of the cluster and class labels.

Completeness Score describes what percentage of data points are members of a given class are in the same clusters. This too is independent of cluster and class labels.